In [2]:
import os

In [3]:
%pwd

'c:\\Users\\Madhav\\Documents\\NLP\\Text-Summarizer\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Madhav\\Documents\\NLP\\Text-Summarizer'

In [44]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [45]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [46]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingesiton_config= DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingesiton_config

In [47]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [48]:
class DataIngestion:
    def __init__(self, config: DataIngestioinConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers=request.urlretrieve(
                url= self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """ 
        zip_file_path: str
        extracts the zip file into the directory
        function returns none"""
        unzip_path= self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

In [49]:
try:
    config= ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2023-12-03 22:29:40,064: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-03 22:29:40,064: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-03 22:29:40,064: INFO: common: Created directory at : artifacts]
[2023-12-03 22:29:40,064: INFO: common: Created directory at : artifacts/data_ingestion]
[2023-12-03 22:29:42,538: INFO: 3934470381: artifacts/data_ingestion/data.zip download with the following info: 
Connection: close
Content-Length: 4187689
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "7512bdb4772b1f93e1f39ca40ae3e8d49af19a9f213c2495814f3fea94e29b41"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9DA4:4A1C:A98BDE:D0A4E0:656CB3FF
Accept-Ranges: bytes
Date: Sun, 03 Dec 2023 16:59:45 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10246-MAA
X-Cache

TypeError: makedirs() got an unexpected keyword argument 'exists_ok'